# Lab 11b
In this lab activities guide you to practice basic skills that were previously covered but can use some refreshing. Some of these are particularly useful for Assignment 3.


## Activity 1
In notebook 3 you have seen numpy's `loadtxt` function. Pull up the docstring again and study the arguments. Note the difference between required and optional arguments (the ones with equal signs). Specifically pay attention to the `unpack` option and the `usecols` options.

In the `data` directory you find a data file with surface properties a model star with a mass 20 times that of the sun, as a function of time. The header describes that the colums are. Open the file and have a look. Recall that you can turn any code cell into a bash cell with the command `%%bash` in the first line. One-line bash commands, such as `cat` or `head` will be understood even without that in a regular python code cell.

Use the `loadtxt` function to read the data in the file.

Make a plot of 
1. The log10 stellar luminosity as a function of the log10 effective temperature. It is customary in astronomy to have the temperature decrease on the abscissa so that cooler, redder objects would be on the right of the diagram. 
2. log10 R as a function of time (star_age).
Save each of these plots to a png image using the `savefig('filename.png')` command.

In [1]:
!head ../data/M20star-surface.dat

# 20Msun surface properties as a function of time; 
# Created with notebook "20Msun-surfacedata.ipynb" on 
# NuGrid server http://wendi1.phys.uvic.ca
# Cols:   star_age [yr], log_L [log10 L/Lsun],    log_Teff [log10 T/K],    log_R [log10 R/Rsun],    log_g [log g [cgs]]
6.250000000000001682e-11 4.190689950571043099e+00 3.502630060967596215e+00 2.613343689802956948e+00 5.122319694711924276e-01
1.125000000000000122e-10 4.201859950931701171e+00 3.505438802160207867e+00 2.613311207598062680e+00 5.122969338809815198e-01
1.632832422125913382e-10 4.201845966017196510e+00 3.505434574592834807e+00 2.613312670275556027e+00 5.122940085259944931e-01
2.242231328677009345e-10 4.201830559252612396e+00 3.505431514088626965e+00 2.613311087901679652e+00 5.122971732737475747e-01
2.973510016538324708e-10 4.201817920746989010e+00 3.505428314856145811e+00 2.613311167113830269e+00 5.122970148494456755e-01
3.851044441971902420e-10 4.201803894054688016e+00 3.505424779181427475e+00 2.613311225117116887e+00 5.122

In [2]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [3]:
star_age,log_L,log_Teff,log_R,log_g = loadtxt('../data/M20star-surface.dat',unpack=True)

In [4]:
figure(115)
(4.551, 4.365)(4.620, 5.089)

FigureCanvasNbAgg()

TypeError: 'tuple' object is not callable

In [5]:
close(115),figure(115)
plot(log_Teff,log_L)
xlim(1.05*max(log_Teff),0.95*min(log_Teff))
xlabel('log Teff');ylabel('log L')
savefig('hrd.png')

FigureCanvasNbAgg()

In [6]:
close(116),figure(116)
plot(star_age,log_R)
xlabel('star_age');ylabel('log R')

FigureCanvasNbAgg()

Text(0, 0.5, 'log R')

## Activity 2
Accordung to fundamental physics the surface temperature and luminosity should be related to the radius of the star according to the [Stefan-Boltzmann law](https://en.wikipedia.org/wiki/Stefan%E2%80%93Boltzmann_law). Specifically for stars it can be cast as
$$
L = 4 \pi R^2  \sigma T_\mathrm{eff}^4
$$
where $\sigma$ is the Stefan-Boltzmann constant, and $R$, $L$ and $T_\mathrm{eff}$ are the star's radius, luminosity and _effective_ temperature. 

Use the radius and temperature read from the file, and the astropy constant and units package to calculate $L$. Compare that Stefan-Boltzmann L to the L from the file. They should be the same! Are they?

Hint: The units package has solar radius etc as units, e.g. `units.Rsun`. Also remember that the `astropy.constants` package provides just that!

In [7]:
from astropy import constants as const
from astropy import units as units

In [8]:
L_sb = lambda R,Teff: 4.*pi*const.sigma_sb * R**2 * Teff**4

In [9]:
R    = 10**log_R * units.Rsun
Teff = 10**log_Teff * units.K

In [10]:
log_diff = log10(L_sb(R,Teff).to('Lsun').value)-log_L

In [11]:
log_diff

array([0.00121078, 0.00121078, 0.00121078, ..., 0.00121078, 0.00121078,
       0.00121078])

The difference is minimal, and a constant offset. Maybe the stellar data uses a different constant? 

## Activity 3
During the hydrogen burning phase the model temperature and lumunosity are within the following ranges: 

* `logTeff` in the range `[4.365,4.551]` and 
* `logL` is in range `[4.620, 5.089]`.

Find the list of indices that refer to data points with Teff and L within these bounds.

How many points are in that list?

Create new short arrays `slogTeff` and `slogL` which contain Teff and L in the range just determined, but only every 50th data point. Use array slicing to accomplish that.

Use the numpy function `savetxt` to write `slogTeff` and `slogL`  as columns into the file 'short-data.dat'. Remember that if `a` is an array, then `a.T` is the transpose. This may be useful here. Also, add a meaningful header with column heads.

In [12]:
ind_bool = (log_L<5.089)*(log_L>4.620)*(log_Teff>4.365)*(log_Teff<4.551)

In [13]:
ind = where(log_L)*ind_bool

In [14]:
ind = where(ind[0]>0)[0]
len(ind)

1003

In [15]:
slogTeff = log_Teff[ind][::50]
slogL    = log_L[ind][::50]

In [16]:
savetxt('short-data.dat',array([slogTeff,slogL]).T,\
        header='Short output every 50th data point for\n\
`logTeff` in the range `[4.365,4.551]` and  \n\
`logL` is in range `[4.620, 5.089]`\n\
logTeff                logL')

In [17]:
cat short-data.dat


# Short output every 50th data point for
# `logTeff` in the range `[4.365,4.551]` and  
# `logL` is in range `[4.620, 5.089]`
# logTeff                logL
4.365895352044131705e+00 4.778872797578517684e+00
4.440417873856842057e+00 4.791107055461132802e+00
4.504649130691265491e+00 4.750145213160442914e+00
4.546822860339725381e+00 4.635673814006468163e+00
4.540056114354413808e+00 4.645491048763978448e+00
4.533677922994431064e+00 4.676020933621453324e+00
4.528547465961421992e+00 4.711178253622526846e+00
4.523318735366378185e+00 4.743515954076496399e+00
4.517283062051287601e+00 4.774034642027519482e+00
4.510313748377282828e+00 4.802403968062966655e+00
4.502054731238749774e+00 4.829617935346174562e+00
4.491318724430080955e+00 4.858250794315693177e+00
4.478074635867191233e+00 4.886561235481562626e+00
4.461306607538522151e+00 4.915069061036878573e+00
4.439728870745694778e+00 4.943996736785328139e+00
4.406968542893594076e+00 4.976111293404159142e+00
4.391632169386393336e+00 4.98569982827506308

In [18]:
close(178);figure(178)
plot(log_Teff[ind],log_L[ind],'-')
plot(slogTeff,slogL,'--')

FigureCanvasNbAgg()